In [ ]:
# dados fin
import yfinance as yf

start = '2021-01-01'
end = '2023-01-01'

data = yf.download('AAPL', 
                   start=start,
                   end=end)

data.head()

data.to_csv('data_/AAPL.csv')

In [34]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd

# Carregando os dados históricos do arquivo CSV
data = pd.read_csv('data_/AAPL.csv')
data.head()

In [ ]:
def calculate_moving_average(data, window):
    return data['Close'].rolling(window=window).mean()

# Vamos testar para diferentes períodos de média móvel (por exemplo, de 5 a 50 dias)
moving_averages = []
for window in range(5, 51):
    moving_averages.append(calculate_moving_average(data, window))


In [ ]:
def calculate_distance_from_moving_average(data, moving_average):
    return abs(data['Close'] - moving_average)

distance_from_moving_averages = []
for moving_average in moving_averages:
    distance_from_moving_averages.append(calculate_distance_from_moving_average(data, moving_average))

# Vamos calcular a média das distâncias para cada período
mean_distances = [dist.mean() for dist in distance_from_moving_averages]


In [ ]:
best_window = mean_distances.index(min(mean_distances)) + 5  # +5 pois começamos a partir da janela 5

print(f'A melhor média móvel é de {best_window} dias.')

In [36]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.linear_model import LinearRegression

# Carregando os dados históricos do arquivo CSV
data = pd.read_csv('data_/AAPL.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Definindo a função para calcular a distância da média móvel simples
def calculate_distance_from_simple_moving_average(data, window):
    moving_average = data['Close'].rolling(window=window).mean()
    return abs(data['Close'] - moving_average)

# Criação de um array de períodos de média móvel a serem testados
window_sizes = np.arange(5, 51)

# Preparação dos dados para validação cruzada
X = data[['Close']]
y = data['Close']

# Definindo o modelo de regressão linear
model = LinearRegression()

# Definindo os parâmetros a serem testados no GridSearch
param_grid = {'window': window_sizes}

# Definindo a estratégia de validação cruzada
tscv = TimeSeriesSplit(n_splits=5)

# Criando o objeto GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)

# Executando a busca pelo melhor período de média móvel
grid_search.fit(X, y)

# Extraindo os resultados
best_window = grid_search.best_params_['window']
best_mse = -grid_search.best_score_

print(f"A melhor média móvel é de {best_window} dias, com MSE de {best_mse:.2f}.")


In [ ]:
# Calculando a melhor média móvel com o período encontrado pelo GridSearch
best_moving_average = data['Close'].rolling(window=best_window).mean()

# Criando colunas para sinal de compra (1), sinal de venda (-1) e sinal neutro (0)
data['Buy_Signal'] = np.where(data['Close'] > best_moving_average, 1, 0)
data['Sell_Signal'] = np.where(data['Close'] < best_moving_average, -1, 0)

# Calculando o lucro e a perda (PnL) da estratégia
data['PnL'] = data['Close'].pct_change() * data['Buy_Signal'].shift(-1)

# Calculando o retorno cumulativo
data['Cumulative_Return'] = (1 + data['PnL']).cumprod()

# Visualizando os resultados
print(data[['Close', 'Buy_Signal', 'Sell_Signal', 'PnL', 'Cumulative_Return']].tail(10))


In [ ]:
# Cálculo do retorno anualizado
days_per_year = 252  # Supondo que há 252 dias úteis em um ano
total_trading_days = data['PnL'].count()
annualized_return = ((data['Cumulative_Return'].iloc[-1]) ** (days_per_year / total_trading_days)) - 1

# Cálculo do risco livre (pode ser ajustado de acordo com a taxa livre de risco atual)
risk_free_rate = 0.03  # Exemplo: 3% ao ano

# Cálculo do índice de Sharpe
daily_returns = data['PnL']
daily_risk_free_rate = (1 + risk_free_rate) ** (1 / days_per_year) - 1
daily_volatility = daily_returns.std()
sharpe_ratio = (daily_returns.mean() - daily_risk_free_rate) / daily_volatility

# Cálculo do drawdown máximo
cumulative_returns = data['Cumulative_Return']
previous_peaks = cumulative_returns.cummax()
drawdowns = cumulative_returns / previous_peaks - 1
max_drawdown = drawdowns.min()

# Visualizando as métricas
print(f"Retorno anualizado: {annualized_return:.2%}")
print(f"Índice de Sharpe: {sharpe_ratio:.2f}")
print(f"Drawdown máximo: {max_drawdown:.2%}")


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Preparação dos dados para o modelo de machine learning
X = data[['Close']].shift(1).dropna()  # Usamos o preço de fechamento do dia anterior como feature
y = data['Buy_Signal'].shift(-1).dropna()  # O sinal de compra do próximo dia é o rótulo

# Divisão dos dados em conjuntos de treinamento e teste
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Criação e treinamento do modelo de Regressão Logística
model = LogisticRegression()
model.fit(X_train, y_train)

# Realizando as previsões para o conjunto de teste
y_pred = model.predict(X_test)

# Avaliando o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
classification_report_output = classification_report(y_test, y_pred)

# Visualizando os resultados
print(f"Acurácia do modelo: {accuracy:.2f}")
print("Relatório de Classificação:\n", classification_report_output)


In [ ]:
from sklearn.svm import SVC

# Criação e treinamento do modelo SVM
model_svm = SVC(kernel='linear', C=1.0)
model_svm.fit(X_train, y_train)

# Realizando as previsões para o conjunto de teste
y_pred_svm = model_svm.predict(X_test)

# Avaliando o desempenho do modelo SVM
accuracy_svm = accuracy_score(y_test, y_pred_svm)
classification_report_output_svm = classification_report(y_test, y_pred_svm)

# Visualizando os resultados do modelo SVM
print(f"Acurácia do modelo SVM: {accuracy_svm:.2f}")
print("Relatório de Classificação do modelo SVM:\n", classification_report_output_svm)


In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Criação e treinamento do modelo de Rede Neural
model_nn = Sequential([
    Dense(64, activation='relu', input_shape=(1,)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model_nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_nn.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# Realizando as previsões para o conjunto de teste
y_pred_nn = model_nn.predict(X_test)

# Avaliando o desempenho do modelo de Rede Neural
accuracy_nn = accuracy_score(y_test, y_pred_nn)
classification_report_output_nn = classification_report(y_test, y_pred_nn)

# Visualizando os resultados do modelo de Rede Neural
print(f"Acurácia do modelo de Rede Neural: {accuracy_nn:.2f}")
print("Relatório de Classificação do modelo de Rede Neural:\n", classification_report_output_nn)


In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Criação e treinamento do modelo ARIMA
model_arima = ARIMA(X_train, order=(5, 1, 0))  # Ordem (p, d, q) ajustável conforme necessário
model_arima_fit = model_arima.fit()

# Realizando as previsões para o conjunto de teste
y_pred_arima = model_arima_fit.forecast(steps=len(X_test))[0]

# Convertendo as previsões para sinais de compra e venda
y_pred_arima_signals = np.where(y_pred_arima > X_test['Close'], 1, -1)

# Avaliando o desempenho do modelo ARIMA
accuracy_arima = accuracy_score(y_test, y_pred_arima_signals)
classification_report_output_arima = classification_report(y_test, y_pred_arima_signals)

# Visualizando os resultados do modelo ARIMA
print(f"Acurácia do modelo ARIMA: {accuracy_arima:.2f}")
print("Relatório de Classificação do modelo ARIMA:\n", classification_report_output_arima)

In [ ]:
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam

# Preparação dos dados para o modelo LSTM
X_train_lstm = X_train.values.reshape(-1, 1, 1)
X_test_lstm = X_test.values.reshape(-1, 1, 1)

# Criação e treinamento do modelo LSTM
model_lstm = Sequential([
    LSTM(64, input_shape=(1, 1)),
    Dense(1, activation='sigmoid')
])

model_lstm.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model_lstm.fit(X_train_lstm, y_train, epochs=50, batch_size=32, verbose=0)

# Realizando as previsões para o conjunto de teste
y_pred_lstm = model_lstm.predict(X_test_lstm)

# Avaliando o desempenho do modelo LSTM
accuracy_lstm = accuracy_score(y_test, y_pred_lstm)
classification_report_output_lstm = classification_report(y_test, y_pred_lstm)

# Visualizando os resultados do modelo LSTM
print(f"Acurácia do modelo LSTM: {accuracy_lstm:.2f}")
print("Relatório de Classificação do modelo LSTM:\n", classification_report_output_lstm)


In [ ]:
!pip install gym

In [ ]:
import numpy as np
import pandas as pd
import gym

class TradingEnv(gym.Env):
    def __init__(self, data):
        super(TradingEnv, self).__init__()
        self.data = data
        self.action_space = gym.spaces.Discrete(3)  # 0: não fazer nada, 1: comprar, 2: vender
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(1,), dtype=np.float32)
        self.reset()

    def reset(self):
        self.current_step = 0
        self.profit = 0
        self.holdings = 0
        self.initial_balance = 1000  # Defina o saldo inicial desejado
        self.balance = self.initial_balance
        return self._next_observation()

    def _next_observation(self):
        return np.array([self.data['Close'].iloc[self.current_step] / self.data['Close'].max()])

    def step(self, action):
        assert self.action_space.contains(action)

        self.current_step += 1

        if action == 1:  # Comprar
            if self.holdings == 0:
                self.holdings = self.balance / self.data['Close'].iloc[self.current_step]
                self.balance = 0

        elif action == 2:  # Vender
            if self.holdings > 0:
                self.balance = self.holdings * self.data['Close'].iloc[self.current_step]
                self.holdings = 0

        # Calcular a recompensa
        current_balance = self.balance + self.holdings * self.data['Close'].iloc[self.current_step]
        self.profit = (current_balance - self.initial_balance) / self.initial_balance

        # Verificar se o episódio terminou (pode adicionar outras condições de término)
        done = self.current_step == len(self.data) - 1

        # Definir a recompensa
        if done:
            reward = self.profit
        else:
            reward = self.profit - 0.001  # Penalidade diária para incentivar ações mais curtas

        return self._next_observation(), reward, done, {}

# Carregar os dados históricos do arquivo CSV
data = pd.read_csv('data_/AAPL.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Criação do ambiente de trading
env = TradingEnv(data)

# Implementação do algoritmo Q-Learning (neste exemplo, não estamos realizando o treinamento, mas você pode implementá-lo)
def q_learning(env, num_episodes=100):
    q_table = np.zeros((len(env.observation_space.high), env.action_space.n))

    for episode in range(num_episodes):
        state = env.reset()
        done = False

        while not done:
            action = np.argmax(q_table[state])
            new_state, reward, done, _ = env.step(action)

            # Atualizar a tabela Q
            q_table[state, action] = (1 - 0.1) * q_table[state, action] + 0.1 * (reward + 0.9 * np.max(q_table[new_state]))

            state = new_state

q_learning(env)

# Teste da estratégia treinada
state = env.reset()
done = False

while not done:
    action = np.argmax(q_table[state])
    state, _, done, _ = env.step(action)

final_profit = env.profit
print(f"Lucro final da estratégia: {final_profit:.2%}")


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Carregar os dados históricos do arquivo CSV
data = pd.read_csv('data_/AAPL.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Pré-processamento dos dados
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data[['Close']])
X, y = [], []
for i in range(len(data_scaled) - 1):
    X.append(data_scaled[i])
    y.append(data_scaled[i + 1][0])

X = np.array(X)
y = np.array(y)

# Divisão dos dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Reshape dos dados para o formato [amostras, tempo, características]
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

# Criação e treinamento do modelo LSTM
model_lstm = Sequential([
    LSTM(64, input_shape=(1, 1)),
    Dense(1)
])

model_lstm.compile(optimizer='adam', loss='mean_squared_error')
model_lstm.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# Realizando as previsões para o conjunto de teste
y_pred_lstm = model_lstm.predict(X_test)

# Transformando as previsões para a escala original
y_pred_lstm = scaler.inverse_transform(y_pred_lstm)
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

# Convertendo as previsões para sinais de compra e venda
y_pred_signals = np.where(y_pred_lstm > y_test, 1, -1)

# Avaliando o desempenho do modelo LSTM
accuracy_lstm = accuracy_score(y_test, y_pred_signals)
classification_report_output_lstm = classification_report(y_test, y_pred_signals)

# Visualizando os resultados do modelo LSTM
print(f"Acurácia do modelo LSTM: {accuracy_lstm:.2f}")
print("Relatório de Classificação do modelo LSTM:\n", classification_report_output_lstm)


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Carregar os dados históricos do arquivo CSV
data = pd.read_csv('data_/AAPL.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Pré-processamento dos dados
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data[['Close']])
X, y = [], []
look_back = 10  # Defina o tamanho da janela temporal
for i in range(len(data_scaled) - look_back):
    X.append(data_scaled[i:i + look_back])
    y.append(data_scaled[i + look_back])

X = np.array(X)
y = np.array(y)

# Divisão dos dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Criação e treinamento do modelo CNN
model_cnn = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(look_back, 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(32, activation='relu'),
    Dense(1)
])

model_cnn.compile(optimizer='adam', loss='mean_squared_error')
model_cnn.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# Realizando as previsões para o conjunto de teste
y_pred_cnn = model_cnn.predict(X_test)

# Transformando as previsões para a escala original
y_pred_cnn = scaler.inverse_transform(y_pred_cnn)
y_test = scaler.inverse_transform(y_test)

# Convertendo as previsões para sinais de compra e venda
y_pred_signals = np.where(y_pred_cnn > y_test, 1, -1)

# Avaliando o desempenho do modelo CNN
accuracy_cnn = accuracy_score(y_test, y_pred_signals)
classification_report_output_cnn = classification_report(y_test, y_pred_signals)

# Visualizando os resultados do modelo CNN
print(f"Acurácia do modelo CNN: {accuracy_cnn:.2f}")
print("Relatório de Classificação do modelo CNN:\n", classification_report_output_cnn)


In [ ]:
!pip install xgboost

In [ ]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Carregar os dados históricos do arquivo CSV
data = pd.read_csv('data_/AAPL.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Preparação dos dados
data['Target'] = np.where(data['Close'].shift(-1) > data['Close'], 1, -1)

X = data[['Close']]
y = data['Target']

# Divisão dos dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Criação e treinamento do modelo XGBoost
model_xgb = XGBClassifier()
model_xgb.fit(X_train, y_train)

# Realizando as previsões para o conjunto de teste
y_pred_xgb = model_xgb.predict(X_test)

# Avaliando o desempenho do modelo XGBoost
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
classification_report_output_xgb = classification_report(y_test, y_pred_xgb)

# Visualizando os resultados do modelo XGBoost
print(f"Acurácia do modelo XGBoost: {accuracy_xgb:.2f}")
print("Relatório de Classificação do modelo XGBoost:\n", classification_report_output_xgb)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Carregar os dados históricos do arquivo CSV
data = pd.read_csv('data_/AAPL.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Preparação dos dados
data['Target'] = np.where(data['Close'].shift(-1) > data['Close'], 1, -1)

X = data[['Close']]
y = data['Target']

# Divisão dos dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Criação e treinamento do modelo Random Forest
model_rf = RandomForestClassifier()
model_rf.fit(X_train, y_train)

# Realizando as previsões para o conjunto de teste
y_pred_rf = model_rf.predict(X_test)

# Avaliando o desempenho do modelo Random Forest
accuracy_rf = accuracy_score(y_test, y_pred_rf)
classification_report_output_rf = classification_report(y_test, y_pred_rf)

# Visualizando os resultados do modelo Random Forest
print(f"Acurácia do modelo Random Forest: {accuracy_rf:.2f}")
print("Relatório de Classificação do modelo Random Forest:\n", classification_report_output_rf)


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Carregar os dados históricos do arquivo CSV
data = pd.read_csv('data_/AAPL.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Preparação dos dados
data['Target'] = np.where(data['Close'].shift(-1) > data['Close'], 1, -1)

X = data[['Close']]
y = data['Target']

# Normalização dos dados
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

# Divisão dos dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Reshape dos dados para o formato adequado
look_back = 10
X_train = np.reshape(X_train, (X_train.shape[0], look_back, 1))
X_test = np.reshape(X_test, (X_test.shape[0], look_back, 1))

# Criação e treinamento do modelo híbrido LSTM + CNN
model_hybrid = Sequential([
    LSTM(64, input_shape=(look_back, 1), return_sequences=True),
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model_hybrid.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_hybrid.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# Realizando as previsões para o conjunto de teste
y_pred_hybrid = model_hybrid.predict_classes(X_test)

# Avaliando o desempenho do modelo híbrido LSTM + CNN
accuracy_hybrid = accuracy_score(y_test, y_pred_hybrid)
classification_report_output_hybrid = classification_report(y_test, y_pred_hybrid)

# Visualizando os resultados do modelo híbrido LSTM + CNN
print(f"Acurácia do modelo híbrido LSTM + CNN: {accuracy_hybrid:.2f}")
print("Relatório de Classificação do modelo híbrido LSTM + CNN:\n", classification_report_output_hybrid)


In [ ]:
import numpy as np
import pandas as pd
import gym
from gym import spaces
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from collections import deque
import random

class TradingEnv(gym.Env):
    def __init__(self, data):
        super(TradingEnv, self).__init__()
        self.data = data
        self.action_space = spaces.Discrete(3)  # 0: não fazer nada, 1: comprar, 2: vender
        self.observation_space = spaces.Box(low=0, high=1, shape=(1,), dtype=np.float32)
        self.reset()

    def reset(self):
        self.current_step = 0
        self.profit = 0
        self.holdings = 0
        self.initial_balance = 1000  # Defina o saldo inicial desejado
        self.balance = self.initial_balance
        return self._next_observation()

    def _next_observation(self):
        return np.array([self.data['Close'].iloc[self.current_step] / self.data['Close'].max()])

    def step(self, action):
        assert self.action_space.contains(action)

        self.current_step += 1

        if action == 1:  # Comprar
            if self.holdings == 0:
                self.holdings = self.balance / self.data['Close'].iloc[self.current_step]
                self.balance = 0

        elif action == 2:  # Vender
            if self.holdings > 0:
                self.balance = self.holdings * self.data['Close'].iloc[self.current_step]
                self.holdings = 0

        # Calcular a recompensa
        current_balance = self.balance + self.holdings * self.data['Close'].iloc[self.current_step]
        self.profit = (current_balance - self.initial_balance) / self.initial_balance

        # Verificar se o episódio terminou (pode adicionar outras condições de término)
        done = self.current_step == len(self.data) - 1

        # Definir a recompensa
        if done:
            reward = self.profit
        else:
            reward = self.profit - 0.001  # Penalidade diária para incentivar ações mais curtas

        return self._next_observation(), reward, done, {}

# Carregar os dados históricos do arquivo CSV
data = pd.read_csv('data_/AAPL.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Criação do ambiente de trading
env = TradingEnv(data)

# Implementação do algoritmo Deep Q-Network (DQN)
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # Fator de desconto
        self.epsilon = 1.0  # Probabilidade de exploração inicial
        self.epsilon_decay = 0.995  # Taxa de decaimento da probabilidade de exploração
        self.epsilon_min = 0.01  # Probabilidade mínima de exploração
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# Configuração do agente DQN
state_size = 1
action_size = 3
batch_size = 32
agent = DQNAgent(state_size, action_size)

# Treinamento do agente DQN
for e in range(100):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    for time in range(500):
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print("Época: {}/{}, Lucro: {:.2f}"
                  .format(e, 100, env.profit))
            break
    if len(agent.memory) > batch_size:
        agent.replay(batch_size)

# Teste da estratégia aprendida
state = env.reset()
state = np.reshape(state, [1, state_size])
done = False
while not done:
    action = agent.act(state)
    next_state, _, done, _ = env.step(action)
    state = np.reshape(next_state, [1, state_size])

final_profit = env.profit
print(f"Lucro final da estratégia: {final_profit:.2f}")
